In [35]:
from docx import Document
from docx.table import Table
from docx.text.paragraph import Paragraph
import re

In [ ]:
def iter_block_items(parent):
    # parent: Document или _Cell (ячейка таблицы)
    parent_elm = parent.element.body if hasattr(parent, "element") else parent._tc
    for child in parent_elm.iterchildren():
        if child.tag.endswith("}p"):
            yield Paragraph(child, parent)
        elif child.tag.endswith("}tbl"):
            yield Table(child, parent)

doc = Document("Проект_ГОСТ_КИТ_на_НК_б.docx")

blocks = []
for block in iter_block_items(doc):
    if isinstance(block, Paragraph):
        text = block.text.strip()
        if text:
            print("P:", text)
            blocks.append("P", text)
    elif isinstance(block, Table):
        print("TABLE:")
        for row in block.rows:
            print([cell.text.strip() for cell in row.cells])
        
        blocks.append()

TABLE:
['ФЕДЕРАЛЬНОЕ АГЕНТСТВО\nПО ТЕХНИЧЕСКОМУ РЕГУЛИРОВАНИЮ И МЕТРОЛОГИИ', 'ФЕДЕРАЛЬНОЕ АГЕНТСТВО\nПО ТЕХНИЧЕСКОМУ РЕГУЛИРОВАНИЮ И МЕТРОЛОГИИ', 'ФЕДЕРАЛЬНОЕ АГЕНТСТВО\nПО ТЕХНИЧЕСКОМУ РЕГУЛИРОВАНИЮ И МЕТРОЛОГИИ']
['', 'ПРЕДВАРИТЕЛЬНЫЙ', '']
['', 'Н А Ц И О Н А Л Ь Н Ы Й', 'ПНСТ\n_________—2024']
['', 'С Т А Н Д А Р Т', 'ПНСТ\n_________—2024']
['', 'Р О С С И Й С К О Й', 'ПНСТ\n_________—2024']
['', 'Ф Е Д Е Р А Ц И И', 'ПНСТ\n_________—2024']
['', '', '']
P: Системы искусственного интеллекта
P: ТЕХНОЛОГИИ КОГНИТИВНЫЕ ИНФОРМАЦИОННЫЕ
P: Термины и определения
P: ISO/TR 9241-810:2020, NEQ
P: Издание официальное
P: ФИО редактора:
P: Духанов Алексей Валентинович
P: тел.моб.: +7 981 143-25-00
P: e-mail: avd_mail@mail.ru
P: Предисловие
P: 1 РАЗРАБОТАН Рабочей группой Национального центра когнитивных разработок Университета ИТМО (НЦКР Университета ИТМО)
P: 2 ВНЕСЕН Техническим комитетом по стандартизации ТК 164 «Искусственный интеллект»
P: 3 УТВЕРЖДЕН И ВВЕДЕН В ДЕЙСТВИЕ Приказом Федерального

In [15]:
def docx_to_text_array(path: str, keep_empty: bool = False) -> list[str]:
    doc = Document(path)
    result: list[str] = []

    # 1) Обычные параграфы
    for p in doc.paragraphs:
        txt = p.text
        if keep_empty or txt.strip():
            result.append(txt)

    # 2) Текст из таблиц (включая вложенные таблицы внутри ячеек)
    def extract_from_table(table):
        for row in table.rows:
            for cell in row.cells:
                # параграфы в ячейке
                for p in cell.paragraphs:
                    txt = p.text
                    if keep_empty or txt.strip():
                        result.append(txt)
                # вложенные таблицы
                for nested in cell.tables:
                    extract_from_table(nested)

    for table in doc.tables:
        extract_from_table(table)

    return result

In [53]:
def extract_terms(lines):
    pattern = re.compile(
        r'^'                      # начало строки
        r'\d+\s+'                 # число + пробелы

        # --- Русская часть: слова и (слова) ---
        r'[А-Яа-яЁё]+'
        r'(?:\s+[А-Яа-яЁё]+|\s+\([А-Яа-яЁё]+\))*'
        r'\s+'                    # пробел перед переводом

        r'(?:'                    # --- далее один из двух форматов перевода ---

            # Вариант A: (английские слова)
            r'\('
            r'[A-Za-z]+'
            r'(?:\s+[A-Za-z]+)*'
            r'\)'

            r'|'                   # или

            # Вариант B: [английские (английские) слова]
            r'\['
            r'[A-Za-z]+'
            r'(?:\s+[A-Za-z]+|\s+\([A-Za-z]+\))*'
            r'\]'

        r')'
    )


    result = [l for l in lines if pattern.match(l)]

    return result

In [74]:
from typing import List, Optional, Tuple
from pypdf import PdfReader


def read_pdf_lines(pdf_path: str) -> List[str]:
    reader = PdfReader(pdf_path)
    lines: List[str] = []
    for page in reader.pages:
        text = page.extract_text() or ""
        for ln in text.splitlines():
            ln = ln.strip()
            if ln:
                lines.append(ln)
    return lines

In [86]:
pdf_lines = read_pdf_lines("gost-r-iso-iec-20546-2021.pdf")
pdf_lines = read_pdf_lines("83677.pdf")

In [88]:
pdf_lines[100:]

['рованная ссылка, то рекомендуется использовать версию этого стандарта с указанным выше годом утверждения',
 '(принятия). Если после утверждения настоящего стандарта в ссылочный стандарт, на который дана датированная',
 'ссылка, внесено изменение, затрагивающее положение, на которое дана ссылка, то это положение рекомендуется',
 'применять без учета данного изменения. Если ссылочный стандарт отменен без замены, то положение, в котором',
 'дана ссылка на него, рекомендуется применять в части, не затрагивающей эту ссылку.',
 '3 Термины и определения',
 'В настоящем стандарте применены термины по ГОСТ Р 59921.0, а также следующие термины с',
 'соответствующими определениями:',
 '3.1',
 'авторизация: Проверка, подтверждение и предоставление прав логического доступа при осу\xad',
 'ществлении субъектами доступа логического доступа.',
 '[ГОСТ Р 57580.1—2017, пункт 3.15]',
 '3.2 архитектура директорий: Система хранения данных, которые могут находиться в разделах',
 'жесткого диска и оператив

# Extract

In [1]:
import os
from parse_pdf_terms_and_clauses import parse_terms_and_clauses_global, write_jsonl
from parse_docx_terms import parse_docx_terms

### Extract - approved

In [2]:
approved_path = "data/docs/approved"
out_terms = "data/extract/approved"

for doc in os.listdir(approved_path):

    pdf_file_path = f"data/docs/approved/{doc}"
    standard_id = ''

    terms, clauses = parse_terms_and_clauses_global(pdf_file_path, standard_id)

    write_jsonl(f"{out_terms}/terms_{doc}.jsonl", terms)
    write_jsonl(f"{out_terms}/clauses_{doc}.jsonl", clauses)

    print(f"PROCESSED: {doc}")
    print(f"> Found {len(terms)} terms")
    print(f"> Found {len(clauses)} clauses")

PROCESSED: gost-r-iso-iec-20546-2021.pdf
> Found 38 terms
> Found 24 clauses
PROCESSED: 83677.pdf
> Found 18 terms
> Found 31 clauses


### Extract - draft

In [3]:
draft_path = "data/docs/draft"
out_path = "data/extract/draft"

for doc in os.listdir(draft_path):

    input_docx = f"{draft_path}/{doc}"
    output_jsonl = f"{out_path}/terms_{doc}.jsonl"

    parse_docx_terms(
        input_docx=input_docx,
        output_jsonl=output_jsonl
    )

Extracted: 60 terms
Saved to: data/extract/draft/terms_Проект_ГОСТ_СППР_на_НК_б.docx.jsonl
Extracted: 72 terms
Saved to: data/extract/draft/terms_Проект_ГОСТ_КИТ_на_НК_б.docx.jsonl


# Load to VectorDB

In [5]:
import chromadb
from OllamaEmbedder import OllamaEmbedder, ollama_embed
from vector_db_utils import upsert_terms, build_retrieval_text, truncate_collection, delete_collections
from parse_pdf_terms_and_clauses import load_jsonl
import os

In [6]:
DB_DIR = "chromadb_volume"
EMBED_MODEL = "nomic-embed-text"
OLLAMA_URL = "http://localhost:11434"

client = chromadb.PersistentClient(path=DB_DIR)
embedder = OllamaEmbedder(model=EMBED_MODEL, base_url=OLLAMA_URL)

### approved - terms

In [7]:
# level -> choices=["national", "international"]

extra_md = {"status": "approved", "level": "national"}

terms_paths = [
    f"data/extract/approved/{name}" for name in os.listdir("data/extract/approved")
    if name.startswith("terms_")
]

truncate_collection(
    client=client,
    collection_name="terms_approved"
)

delete_collections(client=client, collection_name="terms_approved")

for terms_path in terms_paths:
    rows = load_jsonl(terms_path)

    upsert_terms(
        client=client,
        collection_name="terms_approved",
        rows=rows,
        embedder=embedder,
        text_builder=build_retrieval_text,
        extra_md=extra_md,
        batch_size=32,  # embeddings via HTTP, keep moderate
    )   

Коллекция 'terms_approved' удалена


### approved - clauses

In [8]:
# 2) Definitions collection: def only
def_only = lambda r: (r.get("text") or "").strip()

extra_md = {"status": "approved", "level": "national"}

defs_paths = [
    f"data/extract/approved/{name}" for name in os.listdir("data/extract/approved")
    if name.startswith("clauses_")
]

truncate_collection(
    client=client,
    collection_name="clauses_approved"
)

delete_collections(client=client, collection_name="clauses_approved")

for defs_path in defs_paths:
    rows = load_jsonl(defs_path)
    upsert_terms(
        client=client,
        collection_name="clauses_approved",
        rows=rows,
        embedder=embedder,
        text_builder=def_only,
        extra_md=extra_md,
       batch_size=32,
    )

Коллекция 'clauses_approved' удалена


### draft - terms

In [9]:
# level -> choices=["national", "international"]

extra_md = {"status": "draft", "level": "national"}

terms_paths = [
    f"data/extract/draft/{name}" for name in os.listdir("data/extract/draft")
    if name.startswith("terms_")
]

truncate_collection(
    client=client,
    collection_name="terms_draft"
)

delete_collections(client=client, collection_name="terms_draft")

for terms_path in terms_paths:
    rows = load_jsonl(terms_path)
    print(len(rows))
    upsert_terms(
        client=client,
        collection_name="terms_draft",
        rows=rows,
        embedder=embedder,
        text_builder=build_retrieval_text,
        extra_md=extra_md,
        batch_size=32,  # embeddings via HTTP, keep moderate
    )   

Коллекция 'terms_draft' удалена
72
60


# Docs score

In [1]:
from vector_db_utils import retrieve
import chromadb
from OllamaEmbedder import OllamaEmbedder, ollama_embed
from vector_db_utils import upsert_terms, build_retrieval_text, truncate_collection
from parse_pdf_terms_and_clauses import load_jsonl
import os
import numpy as np
from termsim import TermSim
from defsim import DefSim

DB_DIR = "chromadb_volume"
EMBED_MODEL = "nomic-embed-text"
OLLAMA_URL = "http://localhost:11434"

client = chromadb.PersistentClient(path=DB_DIR)
embedder = OllamaEmbedder(model=EMBED_MODEL, base_url=OLLAMA_URL)

In [2]:
DRAFT_DOC_NAME = "Проект_ГОСТ_КИТ_на_НК_б"
#DRAFT_DOC_NAME = "Проект_ГОСТ_СППР_на_НК_б"
APPROVED_DOC_NAME = "83677"#"gost-r-iso-iec-20546-2021"

### TermSim - example

In [3]:
draft_rows = load_jsonl(f"data/extract/draft/terms_{DRAFT_DOC_NAME}.docx.jsonl")
draft_terms = [r["term_ru"] for r in draft_rows]

approved_rows = load_jsonl(f"data/extract/approved/terms_{APPROVED_DOC_NAME}.pdf.jsonl")
approved_terms = [r["term_ru"] for r in approved_rows]

term_score, result = TermSim(
    draft_terms,
    approved_terms
)

print(term_score)

0.04861111111111111


In [5]:
# termsim - pairs
[r for r in result if r["best_term_b"] is not None]

[{'term_a': 'информация', 'best_term_b': 'информация', 'term_score': 1.0},
 {'term_a': 'обработка информации (данных)',
  'best_term_b': 'безопасность информации [данных]',
  'term_score': 0.5},
 {'term_a': 'искусственный интеллект; ИИ (artificial intelligence, AI)',
  'best_term_b': 'система искусственного интеллекта',
  'term_score': 0.5},
 {'term_a': 'знания (в искусственном интеллекте)',
  'best_term_b': 'система искусственного интеллекта',
  'term_score': 0.5},
 {'term_a': 'сильный искусственный интеллект',
  'best_term_b': 'система искусственного интеллекта',
  'term_score': 0.5},
 {'term_a': 'общий искусственный интеллект',
  'best_term_b': 'система искусственного интеллекта',
  'term_score': 0.5}]

### DefSim - example

In [6]:
draft_rows = load_jsonl(f"data/extract/draft/terms_{DRAFT_DOC_NAME}.docx.jsonl")
draft_defintions = [r["definition"].split("П р и м е ч а н и е")[0] for r in draft_rows]

approved_rows = load_jsonl(f"data/extract/approved/terms_{APPROVED_DOC_NAME}.pdf.jsonl")
approved_defintions = [r["definition"].split("П р и м е ч а н и е")[0] for r in approved_rows]

def_score, stats = DefSim(
    draft_defintions,
    approved_defintions,
    ollama_embed=ollama_embed
)

print(def_score)

0.8768839831736185


In [12]:
stats['closest_pairs'][0]

{'a_idx': 57,
 'b_idx': 25,
 'a_text': 'Интерфейс когнитивной информационной системы, одновременно использующий несколько средств обработки разнородной информации, передаваемой пользователем когнитивной информационной системы по различным каналам коммуникации (речевой, текстовый, аудиальный, визуальный, тактильный и так далее).',
 'b_text': 'Рекомендуется использование в процессе работы стандартизированной терминологии (согласно НСИ системы здравоохранения), а также стандартизированных форматов файлов для более точного сбора информации, с возможностью обмена между организациями, осуществляющими СДМ на основе ИИ, унифицированными данными для лечебно-профилактических, научных и иных целей.',
 'sim': 0.9306917190551758}

### ClauseSim - example

In [8]:
draft_rows = load_jsonl(f"data/extract/draft/terms_{DRAFT_DOC_NAME}.docx.jsonl")
draft_defintions = [r["definition"].split("П р и м е ч а н и е")[0] for r in draft_rows]

approved_rows = load_jsonl(f"data/extract/approved/clauses_{APPROVED_DOC_NAME}.pdf.jsonl")
approved_defintions = [r["text"].split("П р и м е ч а н и е")[0] for r in approved_rows]

clau_score, stats = DefSim(
    draft_defintions,
    approved_defintions,
    ollama_embed=ollama_embed
)

print(clau_score)

0.889443963321376


In [11]:
stats["closest_pairs"][0]

{'a_idx': 57,
 'b_idx': 25,
 'a_text': 'Интерфейс когнитивной информационной системы, одновременно использующий несколько средств обработки разнородной информации, передаваемой пользователем когнитивной информационной системы по различным каналам коммуникации (речевой, текстовый, аудиальный, визуальный, тактильный и так далее).',
 'b_text': 'Рекомендуется использование в процессе работы стандартизированной терминологии (согласно НСИ системы здравоохранения), а также стандартизированных форматов файлов для более точного сбора информации, с возможностью обмена между организациями, осуществляющими СДМ на основе ИИ, унифицированными данными для лечебно-профилактических, научных и иных целей.',
 'sim': 0.9306917190551758}

In [13]:
total_score = (term_score + def_score + clau_score) / 3

print(total_score)

0.6049796858687019


# Retrieve

### Approved

In [29]:
terms_approved_collection = client.get_or_create_collection("terms_approved")
draft_collection = client.get_or_create_collection("terms_draft")
clauses_approved_collection = client.get_or_create_collection("clauses_approved")

In [24]:
terms_approved = {}

for row in terms_approved_collection.get()["metadatas"]:
    source_file = row["source_file"]

    if source_file not in terms_approved:
        terms_approved[source_file] = set()
    
    terms_approved[source_file].add(row["term_ru"])

In [25]:
list(terms_approved["83677.pdf"])[:5]

['голосовой помощник',
 'медицинские изделия (изделия медицинского назначения и медицинская техника)',
 'архитектура директорий',
 'программный продукт',
 'система искусственного интеллекта']

### Draft by doc name

In [27]:
draft_doc_name = "Проект_ГОСТ_КИТ_на_НК_б.docx"

draft_docs = draft_collection.get(
    where={"source_file": draft_doc_name}
)["metadatas"]

In [28]:
draft_docs[1]

{'def_len': 141,
 'doc_type': 'term',
 'term_ru': 'информация',
 'source_file': 'Проект_ГОСТ_КИТ_на_НК_б.docx',
 'section_path': '',
 'status': 'draft',
 'term_en': 'information',
 'term_ru_norm': 'информация',
 'level': 'national'}

### Retrive by word - example

In [30]:
query_text = "авторизация"
q_emb = ollama_embed([query_text], model="nomic-embed-text")[0]

res = terms_approved_collection.query(
    query_embeddings=[q_emb],
    n_results=10,
    where={"doc_type": "term"},  # опционально
    include=["documents", "metadatas", "distances"]
)

In [34]:
res["documents"][0][:3]

['TERM_RU: авторизация\nDEF: Проверка, подтверждение и предоставление прав логического доступа при осуществлении субъектами доступа логического доступа. [ГОСТ Р 57580.1—2017, пункт 3.15]',
 'TERM_RU: система дистанционного мониторинга\nDEF: Интеллектуальная информационная система, предназначенная для сбора, хранения, обработки и представления информации, касающейся деятельности медицинских организаций и предоставляемых ими услуг в отношении дистанционного наблюдения за состоянием здоровья пациента.',
 'TERM_RU: информация\nDEF: Сведения (сообщения, данные) независимо от формы их представления. [[1], статья 2]']